## 1. Set up / Authenticate GoogleEarth Engine API

In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='methane-research-analysis') # a Google Eath engine python API is needed see https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api

In [2]:
import datetime
import geemap

## 2. Load the Sentinel-5 Data for Algeria Blowout 

In [3]:
# Define study area coordinates (Algeria blowout example)
lat, lng = 31.8639, 6.1731
#lat, lng = 31, 6
study_area = ee.Geometry.Point([lng, lat])

# Define the date range for methane retrieval
start_date = '2020-01-03'
end_date = '2020-01-04'

In [4]:
# Load a sample image from Sentinel-5P dataset
sample_image = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CH4").first()

# Print the available bands: just EDA good to know our options for Sent-5 (I chose to use the following band for the rest of the notebook: 'CH4_column_volume_mixing_ratio_dry_air_bias_corrected')
print(sample_image.bandNames().getInfo())

['CH4_column_volume_mixing_ratio_dry_air', 'aerosol_height', 'aerosol_optical_depth', 'sensor_azimuth_angle', 'sensor_zenith_angle', 'solar_azimuth_angle', 'solar_zenith_angle', 'CH4_column_volume_mixing_ratio_dry_air_bias_corrected', 'CH4_column_volume_mixing_ratio_dry_air_uncertainty']


In [5]:
# Load Sentinel-5P methane data from start to end date
ch4_dataset = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CH4") \
    .filterDate(start_date, end_date) \
    .filterBounds(study_area)

# Select the methane (CH₄) column-average mixing ratio (bias corrected) band and take the average over the images pulled
ch4_band = 'CH4_column_volume_mixing_ratio_dry_air_bias_corrected'
ch4_image = ch4_dataset.mean().select(ch4_band)

print("Number of images found:", ch4_dataset.size().getInfo()) #just a quick check to see how many images we are pulling (make sure not 0)

Number of images found: 14


## 3. Visualize on GEE map

In [6]:
# Map visualization parameters for methane concentration
ch4_vis_params = {
    'min': 1800,  # Minimum CH₄ concentration (ppb)
    'max': 2000,  # Maximum CH₄ concentration (ppb)
    'palette': ['blue', 'green', 'yellow', 'red']
}

In [7]:
# Create a gee map centered on the study location, in this case where the Algeria blowout occured
map_ch4 = geemap.Map(center=[lat, lng], zoom=8)
map_ch4.addLayer(ch4_image, ch4_vis_params, "Sentinel-5P CH₄")
map_ch4.addLayer(study_area, {"color": "red"}, "Study Location") #add a point for where we know the leak occured in the algeria blowout case

# Add a colorbar legend for methane concentration
map_ch4.add_colorbar(
    vis_params=ch4_vis_params,
    label="Methane Concentration (ppb)",
    orientation="horizontal"
)

# Display the map
map_ch4

Map(center=[31.8639, 6.1731], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…